<center><h1> Lyft: EDA </h1></center>
<img src="https://storage.googleapis.com/kaggle-media/competitions/Lyft-Kaggle/Motion%20Prediction/BP9I1484%20(1).jpg">

# 1. What are we asked to predict in this competition?

So welcome kagglers. In this Lyft competition, we are asked to predict the motion of self driving vehicles. And this knowledge can be used to predict  how cars, cyclists,and pedestrians move in the AV's environment.We have to predict the location of objects agents in the next 50 frames.

## Evaluation Metric: Negative log-likelihood 

We calculate the negative log-likelihood of the ground truth data given the multi-modal predictions. You can get more information [here](https://www.kaggle.com/c/lyft-motion-prediction-autonomous-vehicles/overview/evaluation).
![](https://camo.githubusercontent.com/b3634eea5be5501318957e21086781666018efa1/68747470733a2f2f6c617465782e636f6465636f67732e636f6d2f6769662e6c617465783f25354362675f77686974652532302535436c617267652532304c2532302533442532302d2532302535436c6f6725323070253238785f253742312532432532302535436c646f747325324325323054253744253243253230795f253742312532432532302535436c646f74732532432532305425374425374363253545253742312532432532302535436c646f74732532432532304b253744253243253230253543626172253742782537445f253742312532432532302535436c646f747325324325323054253744253545253742312532432532302535436c646f74732532432532304b253744253243253230253543626172253742792537445f253742312532432532302535436c646f747325324325323054253744253545253742312532432532302535436c646f74732532432532304b253744253239)

# 2. What is the difference between this and previous competition?

[In the previous competition, we were tasked with detecting three dimensional objects that we see normally on the road to teach AV's how to recognize these. ](https://www.kaggle.com/c/3d-object-detection-for-autonomous-vehicles/)
<center><img src="https://storage.googleapis.com/kaggle-media/competitions/Lyft-Kaggle/Kaggle-01.png" width="100%"></center>

# 3. Previous Competition Winners Solutions

<center><img src="https://i.imgur.com/8dUga6i.jpg" width="1000px"></center> 

1. [1st place solution](https://www.kaggle.com/c/3d-object-detection-for-autonomous-vehicles/discussion/122820)
2. [2nd place solution](https://www.kaggle.com/c/3d-object-detection-for-autonomous-vehicles/discussion/123004)
3. [3rd place solution](https://www.kaggle.com/c/3d-object-detection-for-autonomous-vehicles/discussion/117269)

# 4. How files are stored?

The files are stored in the [.zarr file](https://zarr.readthedocs.io/en/stable/) format with Python, which we can easily load using the Level 5 Kit (l5kit for the pip package). Within our training ZARRs, we have the agents, the masks for agents, frames and scenes (which you might recollect from last year) and traffic light faces.

<center><h2>A self driving car in action </h2></center>
Before we dive into the technical details of this kernel, let us watch an interesting video of a self-driving car in action.

In [ ]:
from IPython.display import HTML
HTML('<center><iframe width="700" height="400" src="https://www.youtube.com/embed/tlThdr3O5Qo?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe></center>')

<center><h2> Let's explore the data</h2></center>

# 5. Necessary Imports

## L5Kit is a library which lets you:

* Load driving scenes from zarr files
* Read semantic maps
* Read aerial maps
* Create birds-eye-view (BEV) images which represent a scene around an AV or another vehicle
* Sample data
* Train neural networks
* Visualize results

In [ ]:
!pip install pymap3d==2.1.0
!pip install -U l5kit

In [ ]:
import l5kit, os
from l5kit.rasterization import build_rasterizer
from l5kit.configs import load_config_data
from l5kit.visualization import draw_trajectory, TARGET_POINTS_COLOR
from l5kit.geometry import transform_points
from tqdm import tqdm
from collections import Counter
from l5kit.data import PERCEPTION_LABELS
from prettytable import PrettyTable
from l5kit.data import ChunkedDataset, LocalDataManager
from l5kit.dataset import EgoDataset, AgentDataset
from IPython.display import display, clear_output
import PIL
from IPython.display import display, clear_output
from IPython.display import HTML
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
from PIL import Image
from datetime import time,date
import nltk
import spacy
import re

Now, let's look at the scenes.

# A. Sample Zarr

In [ ]:
os.environ["L5KIT_DATA_FOLDER"] = "../input/lyft-motion-prediction-autonomous-vehicles"
dm = LocalDataManager()
sample_path = '../input/lyft-motion-prediction-autonomous-vehicles/scenes/sample.zarr'
sample_dataset = ChunkedDataset(sample_path)
sample_dataset.open()
print(sample_dataset)

In [ ]:
sample_agents = sample_dataset.agents
sample_agents = pd.DataFrame(sample_agents)
sample_agents.columns = ["data"]; features = ['centroid', 'extent', 'yaw', 'velocity', 'track_id', 'label_probabilities']

for i, feature in enumerate(features):
    sample_agents[feature] = sample_agents['data'].apply(lambda x: x[i])
sample_agents.drop(columns=["data"],inplace=True)
sample_agents.head()

In [ ]:
del sample_agents

# B. Test Zarr

In [ ]:
test_path = '../input/lyft-motion-prediction-autonomous-vehicles/scenes/test.zarr'
test_dataset = ChunkedDataset(test_path)
test_dataset.open()
print(test_dataset)

# C. Train Zarr

In [ ]:
train_path = '../input/lyft-motion-prediction-autonomous-vehicles/scenes/train.zarr'
train_dataset = ChunkedDataset(train_path)
train_dataset.open()
print(train_dataset)

# D. Validation Zarr

In [ ]:
valid_path = '../input/lyft-motion-prediction-autonomous-vehicles/scenes/validate.zarr'
valid_dataset = ChunkedDataset(valid_path)
valid_dataset.open()
print(valid_dataset)

As we have seen the basic exploration above for 4 different ZARR's we will now explore them one by one in detail now

# SEMANTIC VIEW

In [ ]:
cfg = {}
raster_params = {'raster_size':[512,512],
                 'pixel_size':[0.5,0.5],
                 'ego_center':[0.25,0.5],
                 'map_type':'py_semantic',
                 'satellite_map_key': 'aerial_map/aerial_map.png',
                 'semantic_map_key': 'semantic_map/semantic_map.pb',
                 'dataset_meta_key': 'meta.json',
                 'filter_agents_threshold': 0.5}
model_params ={'model_architecture': 'effnetB5',
               'history_num_frames': 0,
               'history_step_size': 1,
               'history_delta_time': 0.1,
               'future_num_frames': 50,
               'future_step_size': 1,
               'future_delta_time': 0.1}
cfg['raster_params'] = raster_params
cfg['model_params'] = model_params
rast = build_rasterizer(cfg,dm)
dataset = EgoDataset(cfg, sample_dataset, rast)
scene_idx = 2
indexes = dataset.get_scene_indices(scene_idx)
images = []

for idx in indexes:
    
    data = dataset[idx]
    im = data["image"].transpose(1, 2, 0)
    im = dataset.rasterizer.to_rgb(im)
    target_positions_pixels = transform_points(data["target_positions"] + data["centroid"][:2], data["world_to_image"])
    center_in_pixels = np.asarray(cfg["raster_params"]["ego_center"]) * cfg["raster_params"]["raster_size"]
    draw_trajectory(im, target_positions_pixels, data["target_yaws"], TARGET_POINTS_COLOR)
    clear_output(wait=True)
    display(PIL.Image.fromarray(im[::-1]))

<h3>As we can see there's a lot of information in this image itself. Let's discuss about this image now.<h3>

* We see around ten roads in the picture. 4 are main and remaining are inside some area like flat or mall or something like that.But we can see vehicles are on the 4 roads.So we can continue our discussion on these 4 roads alone.
* The green line represents the autonomous vehicle in the road and its movement we want to predict in this competition.
* The blue line represents the real vehicle in the road.
* We can also see the vehicles moving.

In [ ]:
cfg = {}
raster_params = {'raster_size':[512,512],
                 'pixel_size':[0.5,0.5],
                 'ego_center':[0.25,0.5],
                 'map_type':'py_semantic',
                 'satellite_map_key': 'aerial_map/aerial_map.png',
                 'semantic_map_key': 'semantic_map/semantic_map.pb',
                 'dataset_meta_key': 'meta.json',
                 'filter_agents_threshold': 0.5}
model_params ={'model_architecture': 'effnetB5',
               'history_num_frames': 0,
               'history_step_size': 1,
               'history_delta_time': 0.1,
               'future_num_frames': 50,
               'future_step_size': 1,
               'future_delta_time': 0.1}
cfg['raster_params'] = raster_params
cfg['model_params'] = model_params
rast = build_rasterizer(cfg,dm)
dataset = EgoDataset(cfg, train_dataset, rast)
scene_idx = 2
indexes = dataset.get_scene_indices(scene_idx)
images = []

for idx in indexes:
    
    data = dataset[idx]
    im = data["image"].transpose(1, 2, 0)
    im = dataset.rasterizer.to_rgb(im)
    target_positions_pixels = transform_points(data["target_positions"] + data["centroid"][:2], data["world_to_image"])
    center_in_pixels = np.asarray(cfg["raster_params"]["ego_center"]) * cfg["raster_params"]["raster_size"]
    draw_trajectory(im, target_positions_pixels, data["target_yaws"], TARGET_POINTS_COLOR)
    clear_output(wait=True)
    display(PIL.Image.fromarray(im[::-1]))

In [ ]:
cfg = {}
raster_params = {'raster_size':[512,512],
                 'pixel_size':[0.5,0.5],
                 'ego_center':[0.25,0.5],
                 'map_type':'py_semantic',
                 'satellite_map_key': 'aerial_map/aerial_map.png',
                 'semantic_map_key': 'semantic_map/semantic_map.pb',
                 'dataset_meta_key': 'meta.json',
                 'filter_agents_threshold': 0.5}
model_params ={'model_architecture': 'effnetB5',
               'history_num_frames': 0,
               'history_step_size': 1,
               'history_delta_time': 0.1,
               'future_num_frames': 50,
               'future_step_size': 1,
               'future_delta_time': 0.1}
cfg['raster_params'] = raster_params
cfg['model_params'] = model_params
rast = build_rasterizer(cfg,dm)
dataset = EgoDataset(cfg, valid_dataset, rast)
scene_idx = 2
indexes = dataset.get_scene_indices(scene_idx)
images = []

for idx in indexes:
    
    data = dataset[idx]
    im = data["image"].transpose(1, 2, 0)
    im = dataset.rasterizer.to_rgb(im)
    target_positions_pixels = transform_points(data["target_positions"] + data["centroid"][:2], data["world_to_image"])
    center_in_pixels = np.asarray(cfg["raster_params"]["ego_center"]) * cfg["raster_params"]["raster_size"]
    draw_trajectory(im, target_positions_pixels, data["target_yaws"], TARGET_POINTS_COLOR)
    clear_output(wait=True)
    display(PIL.Image.fromarray(im[::-1]))

In [ ]:
cfg = {}
raster_params = {'raster_size':[512,512],
                 'pixel_size':[0.5,0.5],
                 'ego_center':[0.25,0.5],
                 'map_type':'py_semantic',
                 'satellite_map_key': 'aerial_map/aerial_map.png',
                 'semantic_map_key': 'semantic_map/semantic_map.pb',
                 'dataset_meta_key': 'meta.json',
                 'filter_agents_threshold': 0.5}
model_params ={'model_architecture': 'effnetB5',
               'history_num_frames': 0,
               'history_step_size': 1,
               'history_delta_time': 0.1,
               'future_num_frames': 50,
               'future_step_size': 1,
               'future_delta_time': 0.1}
cfg['raster_params'] = raster_params
cfg['model_params'] = model_params
rast = build_rasterizer(cfg,dm)
dataset = EgoDataset(cfg, test_dataset, rast)
scene_idx = 2
indexes = dataset.get_scene_indices(scene_idx)
images = []

for idx in indexes:
    
    data = dataset[idx]
    im = data["image"].transpose(1, 2, 0)
    im = dataset.rasterizer.to_rgb(im)
    target_positions_pixels = transform_points(data["target_positions"] + data["centroid"][:2], data["world_to_image"])
    center_in_pixels = np.asarray(cfg["raster_params"]["ego_center"]) * cfg["raster_params"]["raster_size"]
    draw_trajectory(im, target_positions_pixels, data["target_yaws"], TARGET_POINTS_COLOR)
    clear_output(wait=True)
    display(PIL.Image.fromarray(im[::-1]))

# Satellite view

In [ ]:
cfg["raster_params"]["map_type"] = "py_satellite"
rast = build_rasterizer(cfg, dm)
dataset = EgoDataset(cfg, sample_dataset, rast)
scene_idx = 2
indexes = dataset.get_scene_indices(scene_idx)
images = []

for idx in indexes:
    
    data = dataset[idx]
    im = data["image"].transpose(1, 2, 0)
    im = dataset.rasterizer.to_rgb(im)
    target_positions_pixels = transform_points(data["target_positions"] + data["centroid"][:2], data["world_to_image"])
    center_in_pixels = np.asarray(cfg["raster_params"]["ego_center"]) * cfg["raster_params"]["raster_size"]
    draw_trajectory(im, target_positions_pixels, data["target_yaws"], TARGET_POINTS_COLOR)
    clear_output(wait=True)
    display(PIL.Image.fromarray(im[::-1]))

We see that the satellite view gives us better info than the previous view.
We will now see the trajectory path of someother scene.

In [ ]:
cfg["raster_params"]["map_type"] = "py_satellite"
rast = build_rasterizer(cfg, dm)
dataset = EgoDataset(cfg, train_dataset, rast)
scene_idx = 2
indexes = dataset.get_scene_indices(scene_idx)
images = []

for idx in indexes:
    
    data = dataset[idx]
    im = data["image"].transpose(1, 2, 0)
    im = dataset.rasterizer.to_rgb(im)
    target_positions_pixels = transform_points(data["target_positions"] + data["centroid"][:2], data["world_to_image"])
    center_in_pixels = np.asarray(cfg["raster_params"]["ego_center"]) * cfg["raster_params"]["raster_size"]
    draw_trajectory(im, target_positions_pixels, data["target_yaws"], TARGET_POINTS_COLOR)
    clear_output(wait=True)
    display(PIL.Image.fromarray(im[::-1]))

In [ ]:
cfg["raster_params"]["map_type"] = "py_satellite"
rast = build_rasterizer(cfg, dm)
dataset = EgoDataset(cfg, test_dataset, rast)
scene_idx = 2
indexes = dataset.get_scene_indices(scene_idx)
images = []

for idx in indexes:
    
    data = dataset[idx]
    im = data["image"].transpose(1, 2, 0)
    im = dataset.rasterizer.to_rgb(im)
    target_positions_pixels = transform_points(data["target_positions"] + data["centroid"][:2], data["world_to_image"])
    center_in_pixels = np.asarray(cfg["raster_params"]["ego_center"]) * cfg["raster_params"]["raster_size"]
    draw_trajectory(im, target_positions_pixels, data["target_yaws"], TARGET_POINTS_COLOR)
    clear_output(wait=True)
    display(PIL.Image.fromarray(im[::-1]))

In [ ]:
cfg["raster_params"]["map_type"] = "py_satellite"
rast = build_rasterizer(cfg, dm)
dataset = EgoDataset(cfg, valid_dataset, rast)
scene_idx = 2
indexes = dataset.get_scene_indices(scene_idx)
images = []

for idx in indexes:
    
    data = dataset[idx]
    im = data["image"].transpose(1, 2, 0)
    im = dataset.rasterizer.to_rgb(im)
    target_positions_pixels = transform_points(data["target_positions"] + data["centroid"][:2], data["world_to_image"])
    center_in_pixels = np.asarray(cfg["raster_params"]["ego_center"]) * cfg["raster_params"]["raster_size"]
    draw_trajectory(im, target_positions_pixels, data["target_yaws"], TARGET_POINTS_COLOR)
    clear_output(wait=True)
    display(PIL.Image.fromarray(im[::-1]))

In [ ]:
cfg["raster_params"]["map_type"] = "py_satellite"
rast = build_rasterizer(cfg, dm)
dataset = EgoDataset(cfg, sample_dataset, rast)
scene_idx = 34
indexes = dataset.get_scene_indices(scene_idx)
images = []

for idx in indexes:
    
    data = dataset[idx]
    im = data["image"].transpose(1, 2, 0)
    im = dataset.rasterizer.to_rgb(im)
    target_positions_pixels = transform_points(data["target_positions"] + data["centroid"][:2], data["world_to_image"])
    center_in_pixels = np.asarray(cfg["raster_params"]["ego_center"]) * cfg["raster_params"]["raster_size"]
    draw_trajectory(im, target_positions_pixels, data["target_yaws"], TARGET_POINTS_COLOR)
    clear_output(wait=True)
    display(PIL.Image.fromarray(im[::-1]))

From the above satellite view we can see that the pink line is the vehicle's movement. Well let's animate and try to make into video.

In [ ]:
import matplotlib.animation as animation
cfg["raster_params"]["map_type"] = "py_satellite"
rast = build_rasterizer(cfg, dm)
dataset = EgoDataset(cfg, sample_dataset, rast)
scene_idx = 34
indexes = dataset.get_scene_indices(scene_idx)
images = []
fig = plt.figure()
for idx in indexes:
    
    data = dataset[idx]
    im = data["image"].transpose(1, 2, 0)
    im = dataset.rasterizer.to_rgb(im)
    target_positions_pixels = transform_points(data["target_positions"] + data["centroid"][:2], data["world_to_image"])
    center_in_pixels = np.asarray(cfg["raster_params"]["ego_center"]) * cfg["raster_params"]["raster_size"]
    draw_trajectory(im, target_positions_pixels, data["target_yaws"], TARGET_POINTS_COLOR)
    clear_output(wait=True)
    images.append(PIL.Image.fromarray(im[::-1]))
im = plt.imshow(images[0], animated=True)
plt.axis('off')
def animate(i):
    im.set_data(images[i])
ani = animation.FuncAnimation(fig, animate, interval=100, blit=False,
                                repeat_delay=1000)
HTML(ani.to_html5_video())

From the above video we can see the movement of vehicles and movement of our Autonomous Vehicle(Pink path). And also we can see that our AV takes a straight path.

In [ ]:
import imageio
import IPython.display
imageio.mimsave("/tmp/gif.gif", images, duration=0.0001)
IPython.display.Image(filename="/tmp/gif.gif", format='png')

The above gif file gives us better view than the above video

# Aerial Map

In [ ]:
image = cv2.imread('../input/lyft-motion-prediction-autonomous-vehicles/aerial_map/aerial_map.png')
image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
plt.figure(figsize=(32,32))
plt.imshow(image)

The general view of the street us borrowed from [this notebook](https://www.kaggle.com/t3nyks/lyft-working-with-map-api)

In [ ]:
from l5kit.data.map_api import MapAPI
from l5kit.rasterization.rasterizer_builder import _load_metadata

semantic_map_filepath = dm.require(cfg["raster_params"]["semantic_map_key"])
dataset_meta = _load_metadata(cfg["raster_params"]["dataset_meta_key"], dm)
world_to_ecef = np.array(dataset_meta["world_to_ecef"], dtype=np.float64)

map_api = MapAPI(semantic_map_filepath, world_to_ecef)
MAP_LAYERS = ["junction", "node", "segment", "lane"]


def element_of_type(elem, layer_name):
    return elem.element.HasField(layer_name)


def get_elements_from_layer(map_api, layer_name):
    return [elem for elem in map_api.elements if element_of_type(elem, layer_name)]


class MapRenderer:
    
    def __init__(self, map_api):
        self._color_map = dict(drivable_area='#a6cee3',
                               road_segment='#1f78b4',
                               road_block='#b2df8a',
                               lane='#474747')
        self._map_api = map_api
    
    def render_layer(self, layer_name):
        fig = plt.figure(figsize=(10, 10))
        ax = fig.add_axes([0, 0, 1, 1])
        
    def render_lanes(self):
        all_lanes = get_elements_from_layer(self._map_api, "lane")
        fig = plt.figure(figsize=(10, 10))
        ax = fig.add_axes([0, 0, 1, 1])
        for lane in all_lanes:
            self.render_lane(ax, lane)
        return fig, ax
        
    def render_lane(self, ax, lane):
        coords = self._map_api.get_lane_coords(MapAPI.id_as_str(lane.id))
        self.render_boundary(ax, coords["xyz_left"])
        self.render_boundary(ax, coords["xyz_right"])
        
    def render_boundary(self, ax, boundary):
        xs = boundary[:, 0]
        ys = boundary[:, 1] 
        ax.plot(xs, ys, color=self._color_map["lane"], label="lane")
        
        
renderer = MapRenderer(map_api)
fig, ax = renderer.render_lanes()

# Let's Explore Zarr

In [ ]:
print("scenes", sample_dataset.scenes)
print("scenes[0]", sample_dataset.scenes[0])
print("scenes", test_dataset.scenes)
print("scenes[0]", test_dataset.scenes[0])
print("scenes", train_dataset.scenes)
print("scenes[0]", train_dataset.scenes[0])
print("scenes", valid_dataset.scenes)
print("scenes[0]", valid_dataset.scenes[0])

# Credits :
[Lyft: Understanding the data + baseline model by Trigram(@nxrprime)](https://www.kaggle.com/nxrprime/lyft-understanding-the-data-baseline-model)

<center><h1> Thanks for Reading My Kernel </h1></center>
<center><h2> Please upvote if you find it useful </h2></center>